In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = "./data"

In [ ]:
train,test = utils.load_data_csv(DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

# Adding the kink feature
PointResiduals = utils.kink_by_residuals(train)
train['PointResiduals'] = pd.Series(PointResiduals, index=train.index)

PointResidualstest = utils.kink_by_residuals(test)
test['PointResiduals'] = pd.Series(PointResidualstest, index=train.index)


c:\miniconda\envs\py36\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True)

In [ ]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [ ]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values,
          train_part.label.values,
          sample_weight=train_part.weight.values)

In [ ]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

In [ ]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

In [ ]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "sample_submission.csv", index_label="id")